In [ ]:

struct Lens
  f :: Function
end

struct Lens
    state # e
    to  # s -> (e,a)
    from # (e,a...) -> t
end






Dual numbers




In [3]:
struct LensDomain
     m::Int64
end

struct Lens 
    it
    dom::LensDomain
    cod::LensDomain
end

(f::Lens)(x) = f.it(x)



id(A::LensDomain) = Lens(x -> (x, dx -> dx), A, A)
fst(A::LensDomain,B::LensDomain) = Lens(x -> (x[1:A.m] , dx -> vcat( dx , zeros(B.m)  )), oplus(A,B), A)
snd(A::LensDomain,B::LensDomain) = Lens(x -> (x[end-B.m:end] , dx -> vcat( zeros(B.m) , dx )), oplus(A,B), B)
function pair(f::Lens,g::Lens)
    @assert f.dom == g.dom
    x -> begin
    (y, df) = f(x)
    (z, dg) = g(x)
    (vcat(y,z), dq -> df(dq[1:f.cod]) + dg(dq[end-g.cod:end])   )

                        ( x[1:f.dom]  )
    end
end
compose(f::Lens, g::Lens) = Lens(
    x -> begin
         (y, df) = f.it(x)
         (z, fg) = g.it(y)
         (z, df ∘ dg)
         end
    ) 
dup(A::LensDomain) = x -> (vcat(x,x), dxx -> dxx[1:A.m] + dxx[A.m+1:end] )
oplus(f,g) = xy -> begin
                (c, df) = f(xy[1:f.dom.m])
                (d, dg) = g(xy[end-g.dom.m:end  ])
                dfg = dcd -> begin 
                             dx = df(dxd[]) 
                             dy = dg(dcd[])
                             vcat(dx,dy)
                             end
                (vcat(c,d) , dfg )
end

# the lens domain isn't really necessary. I guess it might changed the stage? Dimension is now known before x values rather than at the same time. That's nice
mul(A::LensDomain) = x -> (x[1:A.m] .* x[A.m+1:end] , dx -> vcat( dx .* x[A.m+1:end], dx .* x[1:A.m]  ) )
add(A::LensDomain) = x -> (x[1:A.m] .+ x[A.m+1:end] , dx -> vcat( dx , dx ) ) # sum and dup are dual.


#sum(  )
#fold



sin(A::LensDomain) = x -> (sin.(x) , dx -> cos.(x) .* dx)
cos(A::LensDomain) = x -> (cos.(x) , dx -> -sin.(x) .* dx)
pow(A::LensDomain, n) = x -> (x ^ n, dx -> n * dx .* x ^ (n-1) )



pow (generic function with 1 method)

In [8]:
pow(LensDomain(1), 2)(5)[2](1)

10

In [40]:
vcat([1, 2, 3], [4, 5, 6])

6-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6

In [38]:
[4 5 6]

1×3 Array{Int64,2}:
 4  5  6

Question: 
  Where does allocation happen? presumably earlier in the chain. It would be nice to allocate and mutatitively add in stuff

id might involve copying info then

(x, accrefdx) -> accredx == nothing ? (x, dx -> dx) : (x, dx -> accrefdx += dx)




In [1]:
import Pkg; Pkg.add("LinearMaps")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %                                 ]  16.0 %>                         ]  35.4 %Fetching: [=====================>                   ]  51.5 %             ]  67.3 %==================================>      ]  83.3 %Fetching: [==================================>      ]  84.0 %Fetching: [==================================>      ]  84.8 %

  Resolving package versions...
  Installed LinearMaps ─ v2.7.0
Updating `~/.julia/environments/v1.5/Project.toml`
  [7a12625a] + LinearMaps v2.7.0
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [7a12625a] + LinearMaps v2.7.0


In [ ]:
using LinearMaps

In [ ]:
id(A) = x -> (x, UniformScalingMap(I))
f ̊⊕ g = ((x,y) -> ( , ) = f(x); ( , ) = g(y); (   ,   cat(df, dg))
fst(A) = ( (x,y) ->    )
snd(A) = 

    
    
    

In [14]:
using Catlab
using Catlab.Programs
using Catlab.Theories

In [15]:
@present C(FreeBiproductCategory) begin
  W::Ob
  X::Ob
  Y::Ob
  Z::Ob
  f::Hom(X,Y)
  g::Hom(Y,Z)
  h::Hom(Z,W)
  l::Hom(otimes(Z,W),otimes(Y,X))
  m::Hom(otimes(X,Y),otimes(W,Z))
  n::Hom(otimes(W,Z),otimes(X,Y))
end

Presentation{BiproductCategory,Symbol}(Catlab.Theories.FreeBiproductCategory, (Ob = Catlab.Theories.FreeBiproductCategory.Ob{:generator}[W, X, Y, Z], Hom = Catlab.Theories.FreeBiproductCategory.Hom{:generator}[f, g, h, l, m, n]), Dict(:Z => (:Ob => 4),:f => (:Hom => 1),:l => (:Hom => 4),:m => (:Hom => 5),:n => (:Hom => 6),:W => (:Ob => 1),:h => (:Hom => 3),:X => (:Ob => 2),:Y => (:Ob => 3),:g => (:Hom => 2)…), Pair[])

In [17]:
@program C (w :: X) begin
   x, y = f(w)
   g(x,y)
end

LoadError: MethodError: no method matching iterate(::Catlab.WiringDiagrams.DirectedWiringDiagrams.Port)
Closest candidates are:
  iterate(!Matched::Base.RegexMatchIterator) at regex.jl:552
  iterate(!Matched::Base.RegexMatchIterator, !Matched::Any) at regex.jl:552
  iterate(!Matched::LibGit2.GitBranchIter) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/LibGit2/src/reference.jl:343
  ...

In [ ]:
lift(f) = g -> compose(g,f)
lift2(f) = (g,h) -> compose(pair(g,h), f)


iterate(f::MyFunc{otimes(A, B)}) = iterate((compose(f,fst), compose(f,snd)))


struct Yoneda
    it
end

(f::Yoneda)(g) = Yoneda(compose(f.it, g.it)) 
(f::Yoneda)(x,y) = compose(pair(x.it,y.it),f)
(f::Yoneda)(x,args...) compose
multipair(x,args...) = pair(x,multipair(args))
multipair(x) = x


# Yes, the morphisms may already have a notion of application. And we want a different one.


iterate(f::Yoneda) = Yoneda(compose(f.it, fst), iterate( Yoneda(compose(f.it, snd))))


iterate



In [30]:
struct Fred
end
Base.iterate(x::Fred, state=true) = begin 
     print("hey")
     state ? (1,false) : nothing
end

In [33]:
(x,y) = Fred()
x


heyhey

LoadError: BoundsError: attempt to access Fred
  at index [2]

3

In [6]:
dump(:(
begin
    x,y = f(w)
        g(x,y)
    end
    ))

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 3
      file: Symbol In[6]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Expr
          head: Symbol tuple
          args: Array{Any}((2,))
            1: Symbol x
            2: Symbol y
        2: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol f
            2: Symbol w
    3: LineNumberNode
      line: Int64 4
      file: Symbol In[6]
    4: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol g
        2: Symbol x
        3: Symbol y


In [1]:
1 + 1

2